# Fine-tune Llama 2 in Google Colab
> 🗣️ Large Language Model Course

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne), based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da). Special thanks to Tolga HOŞGÖR for his solution to empty the VRAM.

This notebook runs on a T4 GPU. (Last update: 24 Aug 2023)


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
import huggingface_hub
huggingface_hub.notebook_login()

# **Float16**    -  
 is a half-precision floating-point format that uses 16 bits to represent a number. This means that it can only represent a limited range of numbers with a limited precision. Float16 is typically used in applications where memory bandwidth is a major concern, such as machine learning and graphics processing.

# **(NF4)**
 data type for 4-bit quantization

# 4-bit quantization
is a technique used to reduce the number of bits used to represent the weights and activations of a neural network.

### **example of how 4-bit quantization**
can be used to reduce the memory footprint of a neural network.
 Let's say we have a neural network with 1 million weights. Each weight is currently represented using 32 bits, which means that the total memory footprint of the weights is 32 MB.
 If we use 4-bit quantization to represent the weights, we can reduce the memory footprint of the weights to 1.25 MB. This is a 75% reduction in memory usage.



# **epoch**
 is a single pass of the entire training dataset through the learning algorithm.

# **batch size**
of 32, then the model will process 32 training examples before it updates its parameters.
larger batch size:-

1.   leading to faster training speed
2.   more accurate estimate of the gradient, which guides the model's parameter updates
3.require more memory

**gradient_accumulation_steps**
By accumulating gradients over multiple steps, it effectively simulates a larger batch size without actually increasing the memory requirement. This can improve generalization performance, especially when dealing with small batch sizes.with smaller batch sizes, it reduces the memory consumption per iteration


*   Decreasing gradient_accumulation_steps:
*   2.Faster training
* Reduced memory overhead

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
dataset_name = "cnn_dailymail"
new_model = "llama-2-7b-chat-finetuned1"

# QLoRA parameters
#higher value -generate more coherent and contextually appropriate text. but expensive
lora_r = 64
# more value -->less computational , Improved memory , may  performance degradation
lora_alpha = 16
# for  reduce overfitting. more--->less overfitting ,If too many activations are dropped performance degrad
lora_dropout = 0.1

# bitsandbytes parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# TrainingArguments parameters
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps =1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = 100 #maximum number of steps or iterations allowed for training a model.
warmup_ratio = 0.03

# Group sequences into batches with same length
group_by_length = True
save_steps = 0
logging_steps = 25

# SFT parameters
max_seq_length = None
packing = False
device_map = {"": 0}

# **epoch-**
Number of times the entire training dataset is passed through the model during training
Expose the model to all the training data and allow it to learn from patterns and relationships
# **maxsteps-**
* Maximum number of steps or iterations allowed for training the model

* Control the duration of the training process and prevent overtraining

# **warmup ratio**
might be 0.1 or 0.2, meaning that 10% or 20% of the training steps are used for the warmup phase. During this phase, the **learning rate is gradually increased from a small value to the target learning rat**e. This allows the model to learn from the **data more slowly and gradually**, which can help to prevent overfitting and improve generalization performance.

10,000-->training steps and warmup ratio of 0.2.

* first 2,000 training steps for the warmup phase.
* learning rate will be gradually increased from a small value (e.g., 0.0001) to the target learning rate (e.g., 0.01).
* The remaining 8,000 training steps will then be used with the target learning rate.

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset("cnn_dailymail",'3.0.0',split="train")

dataset

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [ ]:
# dataset['test'][0]

In [ ]:
# dataset['test'][10]

In [ ]:
# dataset['test'][5]

# Gradient checkpointing
 is a memory optimization technique --> reduce the amount of memory that is required to train and run large language models.
Gradient checkpointing is particularly effective for Llama2 models because of their large size and complex architecture.


it is possible to reduce the memory  by up to 50%. This can make it possible to train and run Llama2 models on hardware with less memory, or to train larger models on the same hardware.

# The max_grad_norm
to prevent the gradients from becoming too large. It is typically set to a value between 1 and 2. When the gradients are larger than this value, they are clipped to the maximum norm. This means that the gradients are scaled down so that their norm does not exceed the maximum norm.

# learning rate-->
controls the speed at which a model learns, determines the size of the steps

A higher learning rate means that the model will learn faster, while a lower learning rate means that the model will learn slower.


# weight decay
to prevent overfitting in neural networks. It works by penalizing large weights, which encourages the model to learn more compact and generalizable representations of the data.

# optim="paged_adamw_32bit"
 It specifies the optimizer to be used for the training process, which in this case is a variant of the AdamW optimizer called PagedAdamW.

 # lr_scheduler_type="cosine"
A cosine annealing learning rate scheduler is a technique that gradually decreases the learning rate over the course of training, following a cosine function. This approach starts with a high learning rate to quickly explore the loss landscape and then gradually decreases it to refine the model's parameters and avoid overfitting.

# device_map = {"": 0}
is a configuration setting that specifies the mapping of model components to available devices, typically GPUs or CPUs.

# max_seq_length = None
 indicates that there is no upper limit on the length of input sequences during training. This means that the model can be trained on sequences of arbitrary length, which can be beneficial for tasks that require processing long documents or transcripts.

In [ ]:
# dataset=dataset['train']


In [ ]:
# dataset


configuration object for a machine learning model defines the model's architecture, hyperparameters, and training parameters.

It provides a structured way to store and manage the model's settings, allowing for easier model initialization, saving, and loading.

In [ ]:
0# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
# loading the model with quantization config
 #AutoModelForCausalLM.from_pretrained method is used to load a pre-trained causal language model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

Vanilla model's response

In [ ]:
prompt = """
<User>:(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony, said it was a move toward greater justice. "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. "Indeed, today brings us closer to our shared goals of justice and peace." Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly," she said. Rights group Human Rights Watch welcomed the development. "Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court\'s treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "What\'s objectionable is the attempts to undermine international justice, not Palestine\'s decision to join a treaty to which over 100 countries around the world are members." In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it "strongly" disagreed with the court\'s decision. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. "We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace," it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as "Palestine." While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would "conduct its analysis in full independence and impartiality." The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN\'s Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report.
\n<Summarizer>:
""".strip()

In [ ]:
# Generate a response
generated_text = model.generate(
    input_ids=tokenizer(prompt, return_tensors="pt").input_ids,
    max_new_tokens=300,
    temperature=0.7,
    top_p=0.7,
    num_return_sequences=1,
)

# Decode the generated text
print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<User>:(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's cerem

In [ ]:
def preprocess(data_point):
  return {'document': f"""
<User>: {data_point['article']}
\n<Summarizer>:
""".strip(),
          'summary': f"{data_point['highlights']}"}

In [ ]:
data = dataset.map(preprocess)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    # report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="highlights",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.609300
50,2.670700
75,2.307000
100,2.579500


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

Model's response after training

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony, said it was a move toward greater justice. "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. "Indeed, today brings us closer to our shared goals of justice and peace." Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly," she said. Rights group Human Rights Watch welcomed the development. "Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court\'s treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "What\'s objectionable is the attempts to undermine international justice, not Palestine\'s decision to join a treaty to which over 100 countries around the world are members." In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it "strongly" disagreed with the court\'s decision. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. "We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace," it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as "Palestine." While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would "conduct its analysis in full independence and impartiality." The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN\'s Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report.'
pipe = pipeline(task="summarization", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0])

In [ ]:
prompt = """
<text>: Whether a sign of a good read; or a comment on the \'pulp\' nature of some genres of fiction, the Oxfam second-hand book charts have remained in The Da Vinci Code author\'s favour for the past four years. \n \n Dan Brown has topped Oxfam\'s \'most donated\' list again, his fourth consecutive year. Having sold more than 80 million copies of The Da Vinci Code and had all four of his novels on the New York Times bestseller list in the same week, it\'s hardly surprising that Brown\'s hefty tomes are being donated to charity by readers keen to make some room on their shelves. \n \n Another cult crime writer responsible to heavy-weight hardbacks, Stieg Larsson, is Oxfam\'s \'most sold\' author for the second time in a row. Both the \'most donated\' and \'most sold\' lists are dominated by crime fiction, trilogies and fantasy, with JK Rowling the only female author listed in either of the Top Fives. \n \n Click here or on "View Gallery" to see both charts in pictures ||||| A woman reads a copy of the newly released book \'\'The Lost Symbol\'\' by Dan Brown, at a speed reading book launch event in Sydney, September 15, 2009. REUTERS/Tim Wimborne \n \n SAN FRANCISCO The latest novel from "Da Vinci Code" author Dan Brown, "The Lost Symbol," broke one-day sales records, its publisher and booksellers said. \n \n Readers snapped up over one million hardcover copies across the United States, Canada and the United Kingdom after it was released on Tuesday, said publisher Knopf Doubleday, a division of Random House Inc. \n \n "We are seeing historic, record-breaking sales across all types of our accounts in North America for \'The Lost Symbol," said Sonny Mehta, editor in chief of Knopf Doubleday Publishing Group. Knopf Doubleday is a division of Random House Inc. \n \n Amazon.com Inc, the world\'s largest online retailer, called the book its bestselling first-day adult fiction title ever, including pre-orders. \n \n Barnes & Noble Inc said "The Lost Symbol" broke its previous one-day sales record for adult fiction. \n \n The success of the Dan Brown\'s latest is a boost to publisher Knopf Doubleday and booksellers, which have endured sliding sales in the midst of the recession. \n \n Booksellers have anxiously awaited a popular title that will resonate with readers and fuel the same sort of frenzy seen earlier this decade with the "Harry Potter" series, from author J.K. Rowling. \n \n But the $25 billion domestic book market has wallowed in a slump in recent years as more readers make their purchases online, or forego books altogether for online games and other forms of media and entertainment. \n \n Now, digital books and electronic reading devices such as Amazon\'s Kindle or Sony Corp\'s Reader are seen as both avenues of growth and sources of competition for traditional media, and publishers and booksellers are scrambling to respond. \n \n The highly anticipated book from Brown comes six years after the release of the American novelist\'s last book "The Da Vinci Code". It follows the adventures of Harvard professor Robert Langdon and is set in the secret world of Freemasons in Washington D.C. \n \n "The Da Vinci Code" sold 80 million copies worldwide. It was made into a film starring Tom Hanks that grossed more than $758 million, according to tracking firm Box Office Mojo. \n \n Shares of Amazon closed up $7.15 to $90.70 on the Nasdaq. Amazon was upgraded to a "Buy" rating by Bank of America Merrill Lynch, which cited a resurgence in e-commerce that would benefit the Seattle-based company. \n \n Barnes & Noble closed up 14 cents to $22.04 and book retailer Borders Group Inc closed up 2 cents to $3.29, on the New York Stock Exchange. \n \n (Reporting by Alexandria Sage; editing by Carol Bishopric) ||||| Bestselling author is also the most frequently given away to charity shops \n \n Dan Brown might be one of the world\'s bestselling authors but it turns out that readers aren\'t too keen on keeping his special blend of religious conspiracy and scholarly derring-do on their shelves once they\'ve bought it. \n \n Brown, who has sold more than 81m copies of The Da Vinci Code worldwide, has been revealed as the most donated author to Oxfam\'s 700 high street shops. With just four books to his name – although his long-awaited fifth The Lost Symbol is published next month – Brown did well to see off competition from John Grisham, author of more than 20 and the second-most likely writer to be ditched in a charity shop by readers. \n \n But as secondhand bookshop shelves flood with battered editions of Angels and Demons and Digital Fortress, Brown can comfort himself with the fact that he\'s also Oxfam\'s second most bought author: there are, apparently, still readers out there who have yet to follow the adventures of the dapper symbologist Robert Langdon. There\'s no such consolation for Grisham, whose legal thrillers fail to make Oxfam\'s bestseller charts at all. \n \n "There\'s no question that when you go into the back room of Oxfam shops there are many Dan Brown books," said Oxfam\'s director of trading David McCullough. "But he\'s also very high on the bestseller list so there is a useful recycling exercise going on – it\'s not just people saying \'I\'ve read The Da Vinci Code and now I must get rid of it\'." \n \n Ian Rankin, whose dour, boozy detective John Rebus is no Robert Langdon, tops Oxfam\'s bestseller list, which the charity says is the first ever high-street secondhand bestseller chart. "It\'s always good for an author to know that their books are popular," said the Scottish author, who will unveil a new policeman hero, the teetotal Malcolm Fox, next month. "With Oxfam, it\'s also heartening to realise that each book donated and bought is helping such a worthwhile organisation." \n \n Stephenie Meyer, author of the Twilight series and instigator of myriad teenage crushes courtesy of her sparkly vampire hero Edward Cullen, is also sitting high in Oxfam\'s charts, nestling between Bernard Cornwell and Terry Pratchett. \n \n Margaret Atwood, meanwhile, winner of the Booker prize and author of a host of critically acclaimed works of fiction, scrapes into the list in eighth place, keeping unlikely company with thriller powerhouse James Patterson – currently producing at least eight books a year thanks to a horde of co-writers – and Jodi Picoult, never afraid to jerk a tear or pile on the plot twists. \n \n "We just need to dispel the idea that we are sitting there in Oxfam with only first editions of literary gems – actually we\'ve got shelves of really good fiction," said McCullough. "Waterstone\'s might be more upset than secondhand booksellers," he added, referring to the recent slew of complaints from secondhand booksellers that the charity is stealing their business. \n \n Oxfam, Europe\'s biggest high-street retailer of secondhand books and the third-biggest bookseller in the UK, launched a drive for book donations in May ahead of its first national book festival, Bookfest, in July. Authors including Joanna Trollope, Philip Pullman and Jonathan Coe all lent a hand in shops across the country as part of the festival, and the drive saw book donations rise 40%, with sales up by more than 10%. \n \n Rare books and first editions have also been pouring into shops since May. Ten of the most sought-after editions have raised more than £4,500 for the charity between them. A first edition of Lord of the Rings sold for £800, a first edition of Watership Down brought in £500, Sylvia Plath\'s Ariel sold for £350, Ian Fleming\'s From Russia With Love for £300 and a second printing of Martin Chuzzlewit for £200. \n \n Oxfam, which has more than 130 specialist bookshops and stocks books in almost all of its 700 stores, sells £1.6m-worth of books a month – equivalent, it says, to 50,000 emergency shelters, 64,000 goats or safe water for 2.1 million people. \n \n The most donated authors to Oxfam shops so far this year: \n \n 1. Dan Brown \n \n 2. John Grisham \n \n 3. Ian Rankin \n \n 4. Danielle Steel \n \n 5. Helen Fielding \n \n 6. Stephen King \n \n 7. JK Rowling \n \n 8. Catherine Cookson \n \n 9. Patricia Cornwell \n \n 10. Mills & Boon \n \n The Oxfam shop bestseller list: \n \n 1. Ian Rankin \n \n 2. Dan Brown \n \n 3. Bernard Cornwell \n \n 4. Stephanie Meyer \n \n 5. Terry Pratchett \n \n 6. Khaled Hosseini \n \n 7. Helen Fielding \n \n 8. Margaret Atwood \n \n 9. James Patterson \n \n 10. Jodi Picoult \n \n The top 10 most valuable donated books since May: \n \n 1. JRR Tolkien, Lord of the Rings – first edition, sold for £800 \n \n 2. Don Giovanni sheet music – first edition, sold for £750 \n \n 3. Sowerby\'s Catalogue of Shells – sold for £600 \n \n 4. Richard Adams, Watership Down – first edition, sold for £500 \n \n 5. Handbook of Indian Dances - first edition with hand-blocked prints, sold for £500 \n \n 6. Richmal Crompton, Just William - first edition, sold for £440 \n \n 7. Sylvia Plath, Ariel – first edition, sold for £350 \n \n 8. Ian Fleming, From Russia With Love – first edition, sold for £300 \n \n 9. Charles Dickens, Martin Chuzzlewit – second print, sold for £200 \n \n 10. WE Johns, Biggles in Australia – first edition, sold for £150 ||||| A charity shop is urging people to stop donating The Da Vinci Code after becoming overwhelmed with copies. \n \n The Oxfam shop in Swansea has been receiving an average of one copy of the Dan Brown novel a week for months, leaving them with little room for any other books. \n \n Staff who are struggling to sell copies of the book have put a note up in the store saying they would rather donors hand in their vinyl instead
<summarizer>:
""".strip()

In [ ]:
encoding = tokenizer(prompt, return_tensors="pt").to(device_map)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)